# Conversational Threads

## What I Learned
I learned how to group traces into threads using a unique thread ID, which helps keep all related traces from a conversation together. This feature is useful for tracking chat sessions and multi-turn dialogue workflows.

## Changes in Code
My main changes involved generating a new UUID for thread metadata and using it to organize runs as conversations. I demonstrated the feature by running the notebook multiple times and included an example where retrieval did not find the requested item.

In [ ]:
import uuid
from langsmith import traceable
from langchain_openai import ChatOpenAI

# Generate a thread ID for this conversation
thread_id = str(uuid.uuid4())

@traceable(
    run_type="chain",
    metadata={"thread_id": thread_id}
)
def chat_turn(message: str, history: list) -> str:
    llm = ChatOpenAI(model="gpt-4o-mini")
    full_conversation = history + [{"role": "user", "content": message}]
    response = llm.invoke(full_conversation)
    return response.content

# Simulate a conversation
history = []
response1 = chat_turn("Hello! What is LangSmith?", history)
print(f"Turn 1: {response1}")

history.append({"role": "user", "content": "Hello! What is LangSmith?"})
history.append({"role": "assistant", "content": response1})

response2 = chat_turn("How does it help with debugging?", history)
print(f"Turn 2: {response2}")

print(f"\nThread ID: {thread_id}")

In [ ]:
# Utility functions
def format_messages(messages: list) -> str:
    """Format conversation messages for display"""
    return "\n".join([f"{m['role']}: {m['content']}" for m in messages])

def extract_user_intent(message: str) -> str:
    """Extract user intent from message"""
    # Simple keyword matching
    if "help" in message.lower():
        return "help_request"
    elif "?" in message:
        return "question"
    else:
        return "statement"